In [9]:
import numpy as np


def adjust_prices_single(df):
    df['DisFacPr'] = df['DisFacPr'].fillna(0).add(1).iloc[::-1].cumprod().iloc[::-1]

    # Ajuster les colonnes de prix
    price_columns = ['DlyOpen', 'DlyHigh', 'DlyLow', 'DlyClose']
    for col in price_columns:
        df[f'{col}'] = df[col] / df['DisFacPr']

    return df


def adjust_prices(df):
    # Créer une copie pour éviter de modifier le DataFrame original
    df_adjusted = df.copy()
    df_adjusted = df_adjusted[df_adjusted['DlyCalDt'] > '1973-01-01']

    # Liste des colonnes de prix à ajuster
    price_columns = ['DlyOpen', 'DlyHigh', 'DlyLow', 'DlyClose']

    # Partitionner le DataFrame par PERMNO pour éviter le shuffle excessif
    df_adjusted = df_adjusted.groupby('PERMNO').apply(
        lambda g: g.sort_values('DlyCalDt').assign(
            DisFacPr=lambda x: x['DisFacPr'].fillna(0).add(1)[::-1].cumprod()[::-1]
        )
    )

    # Ajuster les prix en une seule opération
    for col in price_columns:
        df_adjusted[col] = df_adjusted[col] / df_adjusted['DisFacPr']

    return df_adjusted

In [7]:
from src.types import dsf_dtype_dict, dsp500_list_dict
import dask.dataframe as dd
from dask.distributed import LocalCluster
cluster = LocalCluster()
client = cluster.get_client()

file_to_patch = dd.read_csv("../../data/DSF_V2.csv",
                            dtype=dsf_dtype_dict,
                            parse_dates=['DisExDt', 'DlyCalDt', 'DlyPrevDt'],
                            )

In [ ]:
file_updated = adjust_prices(file_to_patch)

In [ ]:
file_updated.to_csv("../../data/dsf_v2_patched.csv")